In [1]:
! apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [2]:
!git clone https://github.com/ArkZ10/DishDash.git

Cloning into 'DishDash'...
remote: Enumerating objects: 261, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 261 (delta 21), reused 5 (delta 3), pack-reused 205
Receiving objects: 100% (261/261), 8.43 MiB | 3.93 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Updating files: 100% (124/124), done.


### Making Data

In [ ]:
import pandas as pd

group = pd.read_csv('/content/DishDash/ML/food-data/Group.csv', sep=';', header=[1])

In [ ]:
group.shape

(998, 43)

In [ ]:
group.iloc[:, 4].dropna()

0      69-447
1      69-775
2     69-1309
3     69-1310
4     69-1311
5     69-1312
6     69-1315
7     69-1317
8     69-1321
9     69-1502
10    69-1580
11      70-25
12     70-239
13     70-240
14     70-775
15    70-1309
16    70-1312
17    70-1314
18    70-1315
19    70-1318
20    70-1322
21    70-1502
22     75-239
23     79-446
24     81-781
25     82-781
26      86-41
Name: SAYUR 1, dtype: object

In [ ]:
group_label = group.columns

In [ ]:
import random
import pandas as pd

def assign_labels_with_random_columns(data):
    num_users = 1250
    num_labels = len(data.columns)

    user_labels = {}

    for user_id in range(1, num_users + 1):
        ratio_a = 0.2

        num_labels_a = int(ratio_a * num_labels)

        # Ensure all columns are considered
        labels_a = [random.sample(data.columns.tolist(), num_labels)[i % num_labels] for i in range(num_labels_a)]

        user_labels[user_id] = {'A': labels_a}

    return user_labels

In [ ]:
num_users = 1250
user_labels = assign_labels_with_random_columns(group)

user_outputs = {}
for user_id, labels in user_labels.items():
    user_outputs[f"user_{user_id}"] = pd.DataFrame({
        f"{user_id}": labels['A'],
    })

# Concatenate the DataFrames
result_df = pd.concat([user_outputs[f"user_{i}"] for i in range(1, num_users + 1)], axis=1)

# Save the result to a CSV file
result_df.to_csv("Random_Group.csv", index_label="User_ID")
result_df.to_excel("Random_Group.xlsx", index_label="User_ID")



In [ ]:
rand_df = pd.read_csv('/content/Random_Group.csv')

In [ ]:
rand_df.shape

(8, 1251)

In [ ]:
rand_df['1'][0]

'Jagung'

In [ ]:
len(group[rand_df['1'][0]].dropna())

14

In [ ]:
num_rows_to_select = int(len(group[rand_df['3'][0]].dropna()) * 0.3)

random_sample = group[rand_df['3'][0]].dropna().sample(n=num_rows_to_select)

In [ ]:
len(group[rand_df['3'][0]].dropna())

88

In [ ]:
def find_food(user, label):
  num_rows_to_select = int(len(group[rand_df[f'{user}'][label]].dropna()) * 0.3)
  random_sample = group[rand_df[f'{user}'][label]].dropna().sample(n=num_rows_to_select)
  return random_sample

In [ ]:
result_df = pd.DataFrame()

for user in range(1, 1251):
    user_column = pd.concat([find_food(user, label) for label in range(0, 8)], axis=0, ignore_index=True)

    result_df[f'{user}'] = user_column

result_df.to_csv("Random_Food.csv", index_label="Row_ID")
result_df.to_excel("Random_Food.xlsx", index_label="Row_ID")

### ONE HOT ENCODE

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
df = pd.read_csv('/content/DishDash/ML/food-data/data_1d.csv', sep=';', header=[1])

In [5]:
df.drop(df.columns[0:2], axis=1, inplace=True)

In [6]:
df.drop(df.columns[3], axis=1, inplace=True)

In [7]:
df.rename(
    columns={
        'ID' : 'food_id',
        'MAKANN' : 'food_name',
        'TEMPAT': 'resto_name'
    }, inplace=True
)

In [8]:
like_df = pd.read_csv('/content/DishDash/ML/user-data/like_data.csv', sep=';')

In [9]:
like_with_food = like_df.merge(df, on='food_id')

In [10]:
like_with_food.drop(like_with_food.columns[3:], axis=1, inplace=True)

In [11]:
like_with_food

,user_id,food_id,like
0,102,01-1164,1
1,1045,01-1164,1
2,1049,01-1164,1
3,1051,01-1164,1
4,1056,01-1164,1
...,...,...,...
92700,887,96-67,1
92701,893,96-67,1
92702,903,96-67,1
92703,917,96-67,1


### MERGING

In [14]:
food_id_data = df['food_id']
food_id_df = pd.DataFrame(food_id_data)

like_data = like_with_food
like_data_df = pd.DataFrame(like_data)

all_combinations_df = pd.DataFrame([(user_id, food_id) for user_id in like_data_df['user_id'].unique() for food_id in food_id_df['food_id']], columns=['user_id', 'food_id'])

merged_df = pd.merge(all_combinations_df, like_data_df, on=['user_id', 'food_id'], how='left')
merged_df['like'] = merged_df['like'].fillna(0).astype(int)

In [15]:
food_like = merged_df.merge(df, on='food_id')

In [16]:
food_like

,user_id,food_id,like,food_name,resto_name
0,102,01-42,1,Ayam,Tukang Ayam
1,1045,01-42,0,Ayam,Tukang Ayam
2,1049,01-42,1,Ayam,Tukang Ayam
3,1051,01-42,0,Ayam,Tukang Ayam
4,1056,01-42,0,Ayam,Tukang Ayam
...,...,...,...,...,...
4183129,1184,96-1565,0,Tongseng,Dialoogi
4183130,188,96-1565,0,Tongseng,Dialoogi
4183131,52,96-1565,0,Tongseng,Dialoogi
4183132,163,96-1565,0,Tongseng,Dialoogi


In [14]:
json_data = food_like.to_json(orient='records')

with open('output_file.json', 'w') as json_file:
    json_file.write(json_data)

### LIKE BASED

In [17]:
data = food_like

In [18]:
user_encoder = LabelEncoder()
food_encoder = LabelEncoder()

data['user_encoded'] = user_encoder.fit_transform(data['user_id'])
data['food_encoded'] = food_encoder.fit_transform(data['food_id'])

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [19]:
embedding_size = 50

def create_model(embedding_size):
  user_input = tf.keras.layers.Input(shape=(1,), name='user_input')
  food_input = tf.keras.layers.Input(shape=(1,), name='food_input')

  user_embedding = tf.keras.layers.Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_size, name='user_embedding')(user_input)
  food_embedding = tf.keras.layers.Embedding(input_dim=len(food_encoder.classes_), output_dim=embedding_size, name='food_embedding')(food_input)

  dot_product = tf.keras.layers.Dot(axes=2)([user_embedding, food_embedding])
  flatten = tf.keras.layers.Flatten()(dot_product)

  output = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)
  model = tf.keras.models.Model(inputs=[user_input, food_input], outputs=output)

  return model

In [20]:
model = create_model(embedding_size)

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=[
        'acc',
        tf.keras.metrics.AUC(name='auc'),
    ])

In [22]:
model.fit([train_data['user_encoded'], train_data['food_encoded']], train_data['like'], epochs=3, batch_size=128, verbose = 1)

Epoch 1/3
26145/26145 [==============================] - 182s 7ms/step - loss: 0.1049 - acc: 0.9778 - auc: 0.7749
Epoch 2/3
26145/26145 [==============================] - 171s 7ms/step - loss: 0.0816 - acc: 0.9778 - auc: 0.8743
Epoch 3/3
26145/26145 [==============================] - 168s 6ms/step - loss: 0.0702 - acc: 0.9778 - auc: 0.9333


In [23]:
test_loss, test_acc, auc = model.evaluate([test_data['user_encoded'], test_data['food_encoded']], test_data['like'])
print(f'Test Accuracy: {test_acc * 100:.2f}%')

26145/26145 [==============================] - 47s 2ms/step - loss: 0.0695 - acc: 0.9775 - auc: 0.9373
Test Accuracy: 97.75%


In [ ]:
model.save("fix_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### PREDICT


In [25]:
data_img = pd.read_csv('/content/preprocess_content_based_data.csv', sep=',')

In [26]:
data_img

,Unnamed: 0.1,Unnamed: 0,food_id,food_name,Label Daging,Label Sayur,Label Buah,Label Kacang,Bumbu,Singkong,...,Label Seafood,Label Tepung,Label Kuah,Label Pedas,Daging Ayam,Daging Kambing,Daging Babi,Daging Sapi,Ikan,image
0,0,0,1.0,Abon,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,https://img-cdn.medkomtek.com/PbrY9X3ignQ8sVuj...
1,1,1,2.0,Abon haruwan,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,https://img-global.cpcdn.com/recipes/cbf330fbd...
2,2,2,3.0,Abon sapi,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,https://img.kurio.network/bBTENwW0CA40Pmyk26Er...
3,3,3,4.0,Acar kuning,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,https://img-global.cpcdn.com/recipes/23fc7d563...
4,4,4,5.0,Agar-agar,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,https://res.cloudinary.com/dk0z4ums3/image/upl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,1609,1609,1610.0,Wortel Rebus,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,https://asset-a.grid.id/crop/0x222:594x690/700...
1610,1610,1610,1611.0,Wortel Segar,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,https://www.astronauts.id/blog/wp-content/uplo...
1611,1611,1611,1612.0,Yangko,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,https://serikatnews.com/wp-content/uploads/202...
1612,1612,1612,1613.0,Yoghurt,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...


In [27]:
new_user_id = 1253
liked_food_id = ['03-1164', '01-1164', '64-1469', '68-540', '69-231', '03-198']

user_encoder.fit(pd.concat([data['user_id'], pd.Series([new_user_id])]))
user_encoder.transform([new_user_id])
np.array([food_encoder.transform(liked_food_id)])

new_user_model = create_model(embedding_size)

new_user_model.compile(optimizer='adam', loss='binary_crossentropy')
new_user_embedding_weights  = new_user_model.get_layer('user_embedding').get_weights()[0]
user_embedding = new_user_model.get_layer('user_embedding').get_weights()[0]

user_embedding /= np.linalg.norm(user_embedding, axis=1, keepdims=True)
new_user_embedding_weights /= np.linalg.norm(new_user_embedding_weights, axis=1, keepdims=True)


In [28]:
def get_image_path(food_id):
    return data_img.loc[data_img['food_id'] == food_id].reset_index(drop=True)['image'].iloc[0]

In [29]:
similarity_scores = cosine_similarity(
    user_embedding,
    new_user_embedding_weights
)[0]

top_similar_users_indices = np.argsort(similarity_scores)[::-1]

liked_items_of_similar_users = data[(data['user_id'].isin(top_similar_users_indices[:6])) & (data['like'] == 1)].drop_duplicates()

randomized_rows = liked_items_of_similar_users.sample(frac=1)

recommended_randomized_food = randomized_rows[['food_id', 'food_name']].values.tolist()

food_id_list = [food_id for food_id, _ in recommended_randomized_food[:6]]
food_name_list = [food_name for _,food_name in recommended_randomized_food[:6]]


food_id = []
resto_id = []
for id in food_id_list:
  parts = id.split('-')
  food_id.append(int(parts[1]))
  resto_id.append(int(parts[0]))

for id, name in zip(food_id, food_name_list):
    image_path = get_image_path(int(id))
    print(f'Rekomendasi: {name}')
    print(f'Food ID: {int(id)}')
    print(f'Image Path: {image_path}')

Rekomendasi: Ayam goreng paha
Food ID: 58
Image Path: https://d1sag4ddilekf6.cloudfront.net/compressed_webp/items/IDITE20210726121341017693/detail/menueditor_item_428cb0ea19e44146a00e1cfbef85faae_1627524568218570621.webp
Rekomendasi: Kacang Gude biji muda
Food ID: 700
Image Path: https://filebroker-cdn.lazada.co.id/kf/S7e8dd05201e2416882c2894863f7f8b6p.jpg
Rekomendasi: Ikan layur segar
Food ID: 545
Image Path: https://ikanwiki.com/gambar/gambar_besar/ikanlay.webp
Rekomendasi: Hati ayam goreng
Food ID: 475
Image Path: https://awsimages.detik.net.id/community/media/visual/2022/02/14/resep-hari-ayam-goreng-ketumbar_43.jpeg?w=1200
Rekomendasi: Soto kudus masakan
Food ID: 1387
Image Path: https://www.masakapahariini.com/wp-content/uploads/2020/02/soto-kudus-7.jpg
Rekomendasi: Mangga manalagi
Food ID: 1000
Image Path: https://asset.kompas.com/crops/lqMd8zhC3HFtV1Vom8TI4oHceOY=/32x20:1000x665/750x500/data/photo/2022/06/15/62a925cdeef37.jpg


In [30]:
food_id

[58, 700, 545, 475, 1387, 1000]

In [31]:
food_name_list

['Ayam goreng paha',
 'Kacang Gude biji muda',
 'Ikan layur segar',
 'Hati ayam goreng',
 'Soto kudus masakan',
 'Mangga manalagi']